# Prepare

In [ ]:
!nvidia-smi

In [ ]:
import torch
from functorch import vmap
x = torch.randn(3)
y = vmap(torch.sin)(x)
assert torch.allclose(y, x.sin())

In [ ]:
import functorch
!python -m xformers.info

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler, StableDiffusionImg2ImgPipeline
import torch
from torch import autocast

In [ ]:
# scheduler.save_pretrained('scheduler/EulerDiscreteScheduler')

In [ ]:
scheduler = EulerDiscreteScheduler.from_pretrained("scheduler/EulerDiscreteScheduler", subfolder="scheduler")

In [ ]:
# remote model path
# model_id = "runwayml/stable-diffusion-v1-5"

# local model path
model_id = "models/stable-diffusion-v1.5"



# Use the Euler scheduler here instead
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id,scheduler=scheduler, revision="fp16", torch_dtype=torch.float16)
# pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, revision="fp16", torch_dtype=torch.float16)
pipe = pipe.to(0)
pipe.enable_attention_slicing()
pipe.safety_checker = (lambda images, clip_input: (images, False))
pipe.device

In [ ]:
# save the model locally after initial download
# pipe.save_pretrained('models/stable-diffusion-v1.5')

In [ ]:
!pip install python-slugify

# RUN

In [ ]:
from glob import glob
input_path = "image_to_image/first_grid"
filename_list = sorted(glob(input_path+"/*", recursive = True))

In [ ]:
from PIL import Image

image_list = []
for curr_filename in filename_list:
    with Image.open(curr_filename) as im:
        im = im.convert("RGB")
        image_list.append(im)

In [ ]:
image_list

In [ ]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

In [ ]:
# make 2x2 grid
grid_2x2 = get_concat_v(get_concat_h(image_list[0], image_list[1]),get_concat_h(image_list[2], image_list[3]))

In [ ]:
width, height = grid_2x2.size

small_grid_2x2 = grid_2x2.resize((512, 512))
small_grid_2x2

In [ ]:
from slugify import slugify
from time import time

# prompt = "low angle 50mm photograph of a majestic bacon-cheesehamburger composed of grilled beef patties and fresh salad and tomato and secret sauce and melted american cheese and sesame seed buns, 4k award winning food photography, trending on artstation"
prompt = "plateau de fromages frais et affinés 999€"

# prompt = "La religieuse est composée de deux choux posés l'un sur l'autre, dont le chou supérieur, qui est censé représenter la tête, est deux fois plus petit.Les choux sont recouverts de fondant du même parfum que la crème pâtissière, et une crème au beurre généralement parfumée au café ou à la vanille permet de tenir la tête."

# prompt = "Religieuse is a French pastry, 999€"

# prompt = "top view picture, of a michelin star restaurant dish, award winning food photography, 4k food picture, french signature dish, silver cutlery on each side of the plate, ornated porcelain plate with floral patterns and gilded meandros, crystal drinking glass with red wine, everything is resting on a marble table, fresh flower arrangement in an ornate vase, ming vase holding flowers"


# prompt = "masterwork oil on canvas still life painting of a cheeseburger banquet by Caravaggio,masterwork oil on canvas still life painting of a cheeseburger feast by Caravaggio, juicy ground beef patty, sesame seed bun, melted american cheese, tomato and pickle slices, trending on artstation, red drape in the background, flower bouquet in ornate flower pot, silver cutlery, ornate vase"


# prompt = "masterwork oil on canvas still life painting of a cheeseburger feast by Caravaggio, the cheeseburger is composed of juicy ground beef patty sesame seed bun melted american cheese tomato pickle slices by Caravaggio, trending on artstation, red drape in the background and flower bouquet in ornate flower pot and silver cutlery and ornate vase by Caravaggio"

# prompt = "masterwork oil on canvas still life painting of a smartphone laying on a table by Caravaggio, smartphone displaying lockscreen by Caravaggio, a amrtphone still life painting by Caravaggio"
# neg_prompt = "blurry, out of focus, frame, framed, new artist, glossy, recent, unknown"
neg_prompt = "frame, blurry"

set_guidance_scale = 7.5

def img2img(prompt, neg_prompt, n_iterations, display_image, in_num_inference_steps, input_seed, input_init_image, strength):
    
    seed_generator = torch.Generator('cuda').manual_seed(input_seed)
    image = None
    for i in range(n_iterations):
        with autocast('cuda'):
          image = pipe(prompt=prompt, init_image = input_init_image,strength = strength, negative_prompt=neg_prompt, height=512, width=512,generator=seed_generator, guidance_scale = set_guidance_scale, num_inference_steps = in_num_inference_steps).images[0]
          image.save(f"images/{slugify(prompt)[:100]}_{i}_{time()}.png")
        if display_image:
            display(image)

    # return image

In [ ]:
curr_seed = 42
# for strength in [0.1, 0.15, 0.2, 0.25, 0.3]:
for strength in [0.3, 0.5, 0.7, 0.9]:
    image = img2img(prompt,neg_prompt, 1, True, 25, curr_seed, small_grid_2x2, strength)